In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time


In [2]:
# Transform: Convert images to tensors & normalize
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load train & test datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 58.4MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.47MB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.63MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class TeacherMLP(nn.Module):
    def __init__(self):
        super(TeacherMLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)  # Large hidden layer
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)  # 10 output classes
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten input
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # No softmax, we use CrossEntropyLoss
        return x

teacher_model = TeacherMLP()


In [4]:
def train_teacher(model, train_loader, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

train_teacher(teacher_model, train_loader)


Epoch 1, Loss: 0.3007126267570486
Epoch 2, Loss: 0.13330838035271841
Epoch 3, Loss: 0.09999120620581736
Epoch 4, Loss: 0.08134710471054464
Epoch 5, Loss: 0.06779255375150901


In [5]:
class StudentMLP(nn.Module):
    def __init__(self):
        super(StudentMLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Smaller hidden layer
        self.fc2 = nn.Linear(128, 10)  # 10 output classes
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten input
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # No softmax, we use distillation loss
        return x

student_model = StudentMLP()


In [7]:
def distillation_loss(student_logits, teacher_logits, labels, alpha=0.5, temperature=3.0):
    """
    Compute the knowledge distillation loss.
    - student_logits: Predictions from student
    - teacher_logits: Predictions from teacher
    - labels: True labels
    - alpha: Balance between soft & hard labels
    - temperature: Softens teacher logits
    """
    soft_targets = nn.functional.softmax(teacher_logits / temperature, dim=1)
    soft_loss = nn.functional.kl_div(
        nn.functional.log_softmax(student_logits / temperature, dim=1), 
        soft_targets, reduction="batchmean"
    ) * (temperature ** 2)

    hard_loss = nn.functional.cross_entropy(student_logits, labels)
    
    return alpha * soft_loss + (1 - alpha) * hard_loss


In [8]:
def train_student(teacher_model, student_model, train_loader, epochs=5):
    criterion = distillation_loss
    optimizer = optim.Adam(student_model.parameters(), lr=0.001)

    teacher_model.eval()  # Teacher model is fixed
    student_model.train()

    for epoch in range(epochs):
        total_loss = 0
        for images, labels in train_loader:
            with torch.no_grad():
                teacher_outputs = teacher_model(images)  # Get teacher predictions
            
            student_outputs = student_model(images)
            loss = criterion(student_outputs, teacher_outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

train_student(teacher_model, student_model, train_loader)


Epoch 1, Loss: 1.3424395718999
Epoch 2, Loss: 0.49816228388977457
Epoch 3, Loss: 0.28418930032169387
Epoch 4, Loss: 0.20802542790452808
Epoch 5, Loss: 0.17380074074845325


In [9]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return correct / total

teacher_acc = evaluate(teacher_model, test_loader)
student_acc = evaluate(student_model, test_loader)

print(f"Teacher Model Accuracy: {teacher_acc * 100:.2f}%")
print(f"Student Model Accuracy: {student_acc * 100:.2f}%")


Teacher Model Accuracy: 96.59%
Student Model Accuracy: 96.42%
